### Import

In [1]:
import pandas as pd
import numpy as np
import gc

import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

In [2]:
order_type1 = [
"1회 이상",
"5회 이상",
"10회 이상",
"15회 이상",
"20회 이상",
"25회 이상",
"30회 이상",
"35회 이상",
"40회 이상",
"45회 이상",
"50회 이상",
"55회 이상",
"60회 이상",
"65회 이상",
"70회 이상",
"75회 이상",
"80회 이상",
"85회 이상",
"90회 이상",
"95회 이상",
"100회 이상",
]

CATEGORICAL_COLUMNS = {
"ID": ("object", []),
"남녀구분코드": ("category", []),
"연령": (
    "ordered",
    ["10대", "20대", "30대", "40대", "50대", "60대", "70대이상"],
),
"Segment": ("category", []),
"가입통신회사코드": ("category", []),
"거주시도명": ("category", []),
"직장시도명": ("category", []),
"_1순위신용체크구분": ("category", []),
"_2순위신용체크구분": ("category", []),
"연회비발생카드수_B0M": ("ordered", ["0개", "1개이상"]),
"Life_Stage": ("category", []),
"대표청구지고객주소구분코드": ("category", []),
"대표청구서수령지구분코드": ("category", []),
"청구서수령방법": ("category", []),
"할인건수_R3M": ("ordered", order_type1),
"할인건수_B0M": ("ordered", order_type1),
"자발한도감액횟수_R12M": ("ordered", ["0회", "1회", "2회", "3회", "4회"]),
"한도증액횟수_R12M": ("ordered", ["0회", "1회이상"]),
"카드론동의여부": ("category", []),
"RV전환가능여부": ("category", []),
"캠페인접촉건수_R12M": ("ordered", order_type1),
"캠페인접촉일수_R12M": (
    "ordered",
    [
        "1일 이상",
        "5일 이상",
        "10일 이상",
        "15일 이상",
        "20일 이상",
        "25일 이상",
        "30일 이상",
    ],
),
"인입횟수_ARS_R6M": ("ordered", order_type1),
"이용메뉴건수_ARS_R6M": ("ordered", order_type1),
"방문횟수_PC_R6M": ("ordered", order_type1),
"방문일수_PC_R6M": ("ordered", order_type1),
"방문횟수_앱_R6M": ("ordered", order_type1),
"OS구분코드": ("category", []),
"_1순위업종": ("category", []),
"_2순위업종": ("category", []),
"_3순위업종": ("category", []),
"_1순위쇼핑업종": ("category", []),
"_2순위쇼핑업종": ("category", []),
"_3순위쇼핑업종": ("category", []),
"_1순위교통업종": ("category", []),
"_2순위교통업종": ("category", []),
"_3순위교통업종": ("category", []),
"_1순위여유업종": ("category", []),
"_2순위여유업종": ("category", []),
"_3순위여유업종": ("category", []),
"_1순위납부업종": ("category", []),
"_2순위납부업종": ("category", []),
"_3순위납부업종": ("category", []),
"최종카드론_금융상환방식코드": ("category", []),
"최종카드론_신청경로코드": ("category", []),
"이용금액대": ("category", []),  # ordered이긴한데... 복잡한 듯
}

### Data Preprocessing(2) : Encoding

In [3]:
train_df = pd.read_parquet("./data/train/merge_train.parquet")
test_df = pd.read_parquet("./data/test/merge_test.parquet")

In [4]:
mem_before_train = train_df.memory_usage(deep=True).sum() / 1024**2
mem_before_test = test_df.memory_usage(deep=True).sum() / 1024**2

object_colums = train_df.select_dtypes(include=['object']).columns

# for col in object_colums:
#     tobe = CATEGORICAL_COLUMNS[col]
#     if tobe[0] == "category":
#         train_df[col] = train_df[col].astype("category")
#         if col != "Segment":
#             test_df[col] = test_df[col].astype("category")
#     elif tobe[0] == "ordered":
#         train_df[col] = train_df[col].astype("category").cat.set_categories(tobe[1], ordered=True)
#         test_df[col] = test_df[col].astype("category").cat.set_categories(tobe[1], ordered=True)

for col in object_colums:
    train_df[col] = train_df[col].astype("category")
    if col != "Segment":
        test_df[col] = test_df[col].astype("category")

print(f"Memory change for train: {mem_before_train} -> {train_df.memory_usage(deep=True).sum() / 1024**2}")
print(f"Memory change for test: {mem_before_test} -> {test_df.memory_usage(deep=True).sum() / 1024**2}")

Memory change for train: 18555.367597579956 -> 11872.13070487976
Memory change for test: 4642.474266052246 -> 3005.819715499878


In [5]:
date_feature = train_df.select_dtypes(include=["datetime64"]).columns.tolist()
feature_cols = train_df.columns.tolist()
feature_cols = [feature for feature in feature_cols if feature not in date_feature + ["Segment"]]

X = train_df[feature_cols].copy()
y = train_df["Segment"].copy()

# 타깃 라벨 인코딩
le_target = LabelEncoder()
y_encoded = le_target.fit_transform(y)

In [6]:
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

X_test = test_df.copy()

encoders = {}  # 각 컬럼별 encoder 저장

for col in categorical_features:
    le_train = LabelEncoder()
    X[col] = le_train.fit_transform(X[col])
    encoders[col] = le_train
    unseen_labels_val = set(X_test[col]) - set(le_train.classes_)
    if unseen_labels_val:
        le_train.classes_ = np.append(le_train.classes_, list(unseen_labels_val))
    X_test[col] = le_train.transform(X_test[col])

In [7]:
gc.collect()

0

### Train

In [8]:
try:
    model = xgb.XGBClassifier(
        tree_method='gpu_hist',  # GPU 모드 설정
        device=0,
        random_state=42,
        enable_categorical=True,
    )
    print("GPU 사용 가능: gpu_hist 모드 적용")
    model.fit(X, y_encoded)
    
except Exception:
    model = xgb.XGBClassifier(
        random_state=42,
        enable_categorical=True,
    )
    print("GPU 사용 불가: CPU 모드 적용")
    model.fit(X, y_encoded)

GPU 사용 가능: gpu_hist 모드 적용
GPU 사용 불가: CPU 모드 적용


In [9]:
model.fit(X, y_encoded)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=True, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...)

### Predict

In [10]:
X_test.drop(columns=['ID'],inplace=True)
X_test.drop(columns=date_feature,inplace=True)

In [11]:
# row-level 예측 수행
y_test_pred = model.predict(X_test)
# 예측 결과를 변환
y_test_pred_labels = le_target.inverse_transform(y_test_pred)

# row 단위 예측 결과를 test_data에 추가
test_data = test_df.copy()  # 원본 유지
test_data["pred_label"] = y_test_pred_labels

### Submission

In [12]:
submission = test_data.groupby("ID")["pred_label"] \
    .agg(lambda x: x.value_counts().idxmax()) \
    .reset_index()

submission.columns = ["ID", "Segment"]

In [ ]:
from datetime import datetime
submission.to_csv(f'./out/{datetime.now().strftime("%y%m%d_%H%M")}_submit.csv',index=False)